## ■ 케글에 개/고양이 상위 6% 안에 드는 것을 목표로 신경망을 구현해보자~

### 1. cifar10 keras 신경망( 책: 미술관에간 딥러닝 책 2장 코드)

In [ ]:
# 필요한 라이브러리 import 하는 코드 
import numpy as np
from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
# to_categorical --> one hot encoding 하는 코드 
# 2 -> [0 0 1 0 0 0 0 0 0 0 0]

import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                            # 텐서 플로우 명령어 필요할 때 tf 대신에 
                            # k 를 쓰겠다라는 의미 


from keras.datasets import cifar10

# 데이터 적제 
NUM_CLASSES = 10 # 클래스 갯수
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) # (50000, 32, 32, 3)
print(y_train.shape) # (50000, 1)

#  0 ~ 1사이로 정규화 한다. 
# 하나의 픽셀(0~255)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 라벨을 10개의 one encoding 생성한다. 
y_train = to_categorical(y_train, NUM_CLASSES)
# to_categorical = np.eyes기능과 유사함.
# y_train의 행렬 데이터를 (50000,1) -----> (50000,NUM_CLASSES) = (50000,10)
print(y_train.shape) # (50000,10)

y_test = to_categorical(y_test, NUM_CLASSES)
# y_test의 행렬 데이터를 (10000,1) --> (10000,10)
print(y_test.shape)

# 모델 만들기 
input_layer = Input((32,32,3)) # 뒤에 batch_size를 넣으므로 1장-> 3차원 데이터를 넣어도 됨.

x = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
# 필터개수 32개, 필터(kernel) 사이즈 3x3
x = BatchNormalization()(x) # 가중치의 정규성을 강제화 시키는 기능 = 배치 정규화
x = LeakyReLU()(x)

x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)


# 완전 연결 계층 
x = Flatten()(x)
print(x.shape)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

# 마지막 출력층
x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x) # (100,10)
# output_layer의 열(10)과 y_train의 열(10)이 같아야 오류가 안뜸.

model = Model(input_layer, output_layer)

# convolution 층 4개, 완전 연결계층 2개인 6층 신경망 

model.summary()

# 모델 훈련
opt = Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
                  # ='binary_crossentropy' (2개만 분류할꺼면 이걸로 써줘도 됨.)
    
model.fit(x_train
          , y_train
          , batch_size=100 # 100장씩 트레이닝 하는 상황.(미니배치)
          , epochs=10
          , shuffle=True
          , validation_data = (x_test, y_test)) 
# 훈련데이터를 100개씩 넣으면서, 테스트데이터도 100개씩 넣으면서 확인함.
# 오버피팅 확인하려고 저렇게 함.

# 훈련이 끝난 신경망에 테스트 데이터를 입력함으로써 신경망 평가를 함,
model.evaluate(x_test, y_test, batch_size=1000) 

Tip. 오차가 안줄고 정확도만 오르고 있는 상황이라면 --> Local minima에 빠져있는 상태   
Tip. 오차가 줄어드는 것에 포커스를 맞추고 관찰하여야함.   

## 예제1. 위의 신경망에서 softmax의 확률백터를 출력하시오!

[0.001 0.02 ...... 0.2] <--- 이미지 한장에 대해서 10개 출력(cifar10의 경우)

[0.1 0.9] <--- 이미지 한장에 대해서 2개 출력(개/고양이일 경우)

### ● 케글에 제출할 때는 테스트 데이터에 대해서 확률백터의 한쪽 확률을 제출해야 된다.

####  위에 코드 마지막에 preds = model.predict(x_test) 를 마지막에 추가해 주면 됨.

### ※ 설명 : preds 는 [10000,10] 크기의 배열입니다. 즉 샘플마다 10개의 클래스 확률을 담은 벡터가 반환합니다.

In [ ]:
# 필요한 라이브러리 import 하는 코드 
# 현재 코딩시간을 감소하여 빨리 확인하려고 1000장씩 줄인상황.

import numpy as np
from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
# to_categorical --> one hot encoding 하는 코드 
# 2 -> [0 0 1 0 0 0 0 0 0 0 0]

import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                            # 텐서 플로우 명령어 필요할 때 tf 대신에 
                            # k 를 쓰겠다라는 의미 


from keras.datasets import cifar10

# 데이터 적제 
NUM_CLASSES = 10 # 클래스 갯수
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) # (50000, 32, 32, 3)
print(y_train.shape) # (50000, 1)

#  0 ~ 1사이로 정규화 한다. 
# 하나의 픽셀(0~255)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 라벨을 10개의 one encoding 생성한다. 
y_train = to_categorical(y_train, NUM_CLASSES)
# to_categorical = np.eyes기능과 유사함.
# y_train의 행렬 데이터를 (50000,1) -----> (50000,NUM_CLASSES) = (50000,10)
print(y_train.shape) # (50000,10)

y_test = to_categorical(y_test, NUM_CLASSES)
# y_test의 행렬 데이터를 (10000,1) --> (10000,10)
print(y_test.shape)

x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

# 모델 만들기 
input_layer = Input((32,32,3)) # 뒤에 batch_size를 넣으므로 1장-> 3차원 데이터를 넣어도 됨.

x = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
# 필터개수 32개, 필터(kernel) 사이즈 3x3
x = BatchNormalization()(x) # 가중치의 정규성을 강제화 시키는 기능 = 배치 정규화
x = LeakyReLU()(x)

x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)


# 완전 연결 계층 
x = Flatten()(x)
print(x.shape)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

# 마지막 출력층
x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x) # (100,10)
# output_layer의 열(10)과 y_train의 열(10)이 같아야 오류가 안뜸.

model = Model(input_layer, output_layer)

# convolution 층 4개, 완전 연결계층 2개인 6층 신경망 

model.summary()

# 모델 훈련
opt = Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
                  # ='binary_crossentropy' (2개만 분류할꺼면 이걸로 써줘도 됨.)
    
model.fit(x_train
          , y_train
          , batch_size=100 # 100장씩 트레이닝 하는 상황.(미니배치)
          , epochs=1
          , shuffle=True
          , validation_data = (x_test, y_test)) 
# 훈련데이터를 100개씩 넣으면서, 테스트데이터도 100개씩 넣으면서 확인함.
# 오버피팅 확인하려고 저렇게 함.

# 훈련이 끝난 신경망에 테스트 데이터를 입력함으로써 신경망 평가를 함,
model.evaluate(x_test, y_test, batch_size=1000) 

preds = model.predict(x_test)
print(preds)

#### ※ 설명 : preds는 [10000,10] 크기의 배열입니다. 즉 샘플마다 10개의 클래스 확률을 담은 벡터가 반환됩니다.

## 예제2. 위에 훈련된 신경망에서 출력된 테스트 데이터의 확률 벡터에서 가장 큰 원소의 인덱스 번호가 무엇이지 출력해서 아래의 클래스 중에 무엇으로 예측했는지 확인하시오!

1. 비행기
2. 자동차
3. 새
4. 고양이
5. 사슴
6. 개
7. 개구리
8. 말
9. 배
10. 트럭

In [ ]:
# 기존 코드에서 맨밑에 이 코드만 추가해 주면됨.

CLASSES = np.array(['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말', '배', '트럭'])
preds = model.predict(x_test) # 일단 전체 말고 하나만 확인
preds_value = CLASSES[np.argmax(preds, axis = -1)][2]
actual_value = CLASSES[np.argmax(y_test, axis = -1)][2]
print(preds_value)
print(actual_value)

# 설명 : 여기서 axis = -1 은 마지막 차원(클래스 차원)으로 배열을 압축해라는 뜻.
# 그래서 (10000,10) ----> (10000,1)

In [ ]:
# 필요한 라이브러리 import 하는 코드 
# 현재 코딩시간을 감소하여 빨리 확인하려고 1000장씩 줄인상황.

import numpy as np
from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
# to_categorical --> one hot encoding 하는 코드 
# 2 -> [0 0 1 0 0 0 0 0 0 0 0]

import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                            # 텐서 플로우 명령어 필요할 때 tf 대신에 
                            # k 를 쓰겠다라는 의미 


from keras.datasets import cifar10

# 데이터 적제 
NUM_CLASSES = 10 # 클래스 갯수
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) # (50000, 32, 32, 3)
print(y_train.shape) # (50000, 1)

#  0 ~ 1사이로 정규화 한다. 
# 하나의 픽셀(0~255)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 라벨을 10개의 one encoding 생성한다. 
y_train = to_categorical(y_train, NUM_CLASSES)
# to_categorical = np.eyes기능과 유사함.
# y_train의 행렬 데이터를 (50000,1) -----> (50000,NUM_CLASSES) = (50000,10)
print(y_train.shape) # (50000,10)

y_test = to_categorical(y_test, NUM_CLASSES)
# y_test의 행렬 데이터를 (10000,1) --> (10000,10)
print(y_test.shape)

# 코드만 되는지 안되는지 확인만 해볼거라 1000개씩 일단 가져옴.
x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

# 모델 만들기 
input_layer = Input((32,32,3)) # 뒤에 batch_size를 넣으므로 1장-> 3차원 데이터를 넣어도 됨.

x = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
# 필터개수 32개, 필터(kernel) 사이즈 3x3
x = BatchNormalization()(x) # 가중치의 정규성을 강제화 시키는 기능 = 배치 정규화
x = LeakyReLU()(x)

x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)


# 완전 연결 계층 
x = Flatten()(x)
print(x.shape)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

# 마지막 출력층
x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x) # (100,10)
# output_layer의 열(10)과 y_train의 열(10)이 같아야 오류가 안뜸.

model = Model(input_layer, output_layer)

# convolution 층 4개, 완전 연결계층 2개인 6층 신경망 

model.summary()

# 모델 훈련
opt = Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
                  # ='binary_crossentropy' (2개만 분류할꺼면 이걸로 써줘도 됨.)
    
model.fit(x_train
          , y_train
          , batch_size=100 # 100장씩 트레이닝 하는 상황.(미니배치)
          , epochs=1
          , shuffle=True
          , validation_data = (x_test, y_test)) 
# 훈련데이터를 100개씩 넣으면서, 테스트데이터도 100개씩 넣으면서 확인함.
# 오버피팅 확인하려고 저렇게 함.

# 훈련이 끝난 신경망에 테스트 데이터를 입력함으로써 신경망 평가를 함,
model.evaluate(x_test, y_test, batch_size=1000) 


CLASSES = np.array(['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말', '배', '트럭'])
preds = model.predict(x_test) # 일단 전체 말고 하나만 확인
preds_value = CLASSES[np.argmax(preds, axis = -1)][2]
actual_value = CLASSES[np.argmax(y_test, axis = -1)][2]
print(preds_value)
print(actual_value)

## 점심시간 문제 : convolution층을 하나 더넣고 fully connect 층도 하나 더 넣어서 10에폭으로 모델을 생성하고 생선된 모델을 아래의 명령어로 내리고 검사받으세요~

In [ ]:
# 필요한 라이브러리 import 하는 코드 
# 현재 코딩시간을 감소하여 빨리 확인하려고 1000장씩 줄인상황.

import numpy as np
from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation, MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
# to_categorical --> one hot encoding 하는 코드 
# 2 -> [0 0 1 0 0 0 0 0 0 0 0]

import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                            # 텐서 플로우 명령어 필요할 때 tf 대신에 
                            # k 를 쓰겠다라는 의미 


from keras.datasets import cifar10

# 데이터 적제 
NUM_CLASSES = 10 # 클래스 갯수
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) # (50000, 32, 32, 3)
print(y_train.shape) # (50000, 1)

#  0 ~ 1사이로 정규화 한다. 
# 하나의 픽셀(0~255)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 라벨을 10개의 one encoding 생성한다. 
y_train = to_categorical(y_train, NUM_CLASSES)
# to_categorical = np.eyes기능과 유사함.
# y_train의 행렬 데이터를 (50000,1) -----> (50000,NUM_CLASSES) = (50000,10)
print(y_train.shape) # (50000,10)

y_test = to_categorical(y_test, NUM_CLASSES)
# y_test의 행렬 데이터를 (10000,1) --> (10000,10)
print(y_test.shape)


# 모델 만들기 
input_layer = Input((32,32,3)) # 뒤에 batch_size를 넣으므로 1장-> 3차원 데이터를 넣어도 됨.

x = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
# 필터개수 32개, 필터(kernel) 사이즈 3x3
x = BatchNormalization()(x) # 가중치의 정규성을 강제화 시키는 기능 = 배치 정규화
x = LeakyReLU()(x)

x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# 추가된 convolution 계층
x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# 완전 연결 계층 
x = Flatten()(x)
print(x.shape)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

# 추가된 완전 연결 계층
x = Dense(100)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

# 마지막 출력층
x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x) # (100,10)
# output_layer의 열(10)과 y_train의 열(10)이 같아야 오류가 안뜸.

model = Model(input_layer, output_layer)

# convolution 층 4개, 완전 연결계층 2개인 6층 신경망 

model.summary()

# 모델 훈련
opt = Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
                  # ='binary_crossentropy' (2개만 분류할꺼면 이걸로 써줘도 됨.)
    
model.fit(x_train
          , y_train
          , batch_size=100 # 100장씩 트레이닝 하는 상황.(미니배치)
          , epochs=10
          , shuffle=True
          , validation_data = (x_test, y_test)) 
# 훈련데이터를 100개씩 넣으면서, 테스트데이터도 100개씩 넣으면서 확인함.
# 오버피팅 확인하려고 저렇게 함.

# 훈련이 끝난 신경망에 테스트 데이터를 입력함으로써 신경망 평가를 함,
model.evaluate(x_test, y_test, batch_size=1000) 


CLASSES = np.array(['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말', '배', '트럭'])
preds = model.predict(x_test) # 일단 전체 말고 하나만 확인
preds_value = CLASSES[np.argmax(preds, axis = -1)][2]
actual_value = CLASSES[np.argmax(y_test, axis = -1)][2]

model.save_weights('cifar10_v1.h5') # 이렇게 쓰면 working 디렉토리에 저장됨.

## 예제3. 훈련이 다된 신경망에 테스트 데이터를 한번에 넣어서 평가하는 코드를 구현하시오!

In [ ]:
# 전체코드에서 이 부분만 수정하면 됨.

model.fit(x_train
          , y_train
          , batch_size=100 # 100장씩 트레이닝 하는 상황.(미니배치)
          , epochs=10
          , shuffle=True, validation_split = 0.25) 


# 케글에서는 test_data에 라벨이 없으므로  validation_data 부분을 뺌.(오버피팅을 확인할 수 없다.)
# validation_split : 훈련할 때 훈련 데이터의 일부를 테스트 데이터(25%)로 쓰면서, 오버피팅이 일어나는지 확인하겠다.

model.evaluate(x_test, y_test, batch_size=1000)

In [ ]:
# 수정한 전체 코드(Full code)

# 필요한 라이브러리 import 하는 코드 
# 현재 코딩시간을 감소하여 빨리 확인하려고 1000장씩 줄인상황.

import numpy as np
from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
# to_categorical --> one hot encoding 하는 코드 
# 2 -> [0 0 1 0 0 0 0 0 0 0 0]

import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                            # 텐서 플로우 명령어 필요할 때 tf 대신에 
                            # k 를 쓰겠다라는 의미 


from keras.datasets import cifar10

# 데이터 적제 
NUM_CLASSES = 10 # 클래스 갯수
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) # (50000, 32, 32, 3)
print(y_train.shape) # (50000, 1)

#  0 ~ 1사이로 정규화 한다. 
# 하나의 픽셀(0~255)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 라벨을 10개의 one encoding 생성한다. 
y_train = to_categorical(y_train, NUM_CLASSES)
# to_categorical = np.eyes기능과 유사함.
# y_train의 행렬 데이터를 (50000,1) -----> (50000,NUM_CLASSES) = (50000,10)
print(y_train.shape) # (50000,10)

y_test = to_categorical(y_test, NUM_CLASSES)
# y_test의 행렬 데이터를 (10000,1) --> (10000,10)
print(y_test.shape)


# 모델 만들기 
input_layer = Input((32,32,3)) # 뒤에 batch_size를 넣으므로 1장-> 3차원 데이터를 넣어도 됨.

x = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
# 필터개수 32개, 필터(kernel) 사이즈 3x3
x = BatchNormalization()(x) # 가중치의 정규성을 강제화 시키는 기능 = 배치 정규화
x = LeakyReLU()(x)

x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# 추가된 convolution 계층
x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# 완전 연결 계층 
x = Flatten()(x)
print(x.shape)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

# 추가된 완전 연결 계층
x = Dense(100)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

# 마지막 출력층
x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x) # (100,10)
# output_layer의 열(10)과 y_train의 열(10)이 같아야 오류가 안뜸.

model = Model(input_layer, output_layer)

# convolution 층 4개, 완전 연결계층 2개인 6층 신경망 

model.summary()

# 모델 훈련
opt = Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
                  # ='binary_crossentropy' (2개만 분류할꺼면 이걸로 써줘도 됨.)
    
model.fit(x_train
          , y_train
          , batch_size=100 # 100장씩 트레이닝 하는 상황.(미니배치)
          , epochs=10
          , shuffle=True, validation_split=0.25) 

# 훈련데이터를 100개씩 넣으면서, 테스트데이터도 100개씩 넣으면서 확인함.
# 오버피팅 확인하려고 저렇게 함.
# 훈련이 끝난 신경망에 테스트 데이터를 입력함으로써 신경망 평가를 함,
model.evaluate(x_test, y_test, batch_size=1000) 


CLASSES = np.array(['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말', '배', '트럭'])
preds = model.predict(x_test) # 일단 전체 말고 하나만 확인
preds_value = CLASSES[np.argmax(preds, axis = -1)][2]
actual_value = CLASSES[np.argmax(y_test, axis = -1)][2]

print (preds_value)
print (actual_value)

model.save_weights('d:\\data\\cifar10_v1.h5') # 저장위치를 설정할 수도 있다.

## 예제4. 훈련 데이터 정확도 74% 의 모델 cifar10_v1.h5 모델을 불러와서 cifar10 테스트 데이터 한장을 잘 맞추는지 확인하시오!

훈련은 시키지 말고 모델을 바로 불러와서 사진 한장을 잘 맞추는지 확인하시오! (아래의 코드를 참고하시오!)

In [ ]:
# 힌트
# 모델을 불러오는 코드
self.model = self.build_model()
    if train:
        self.model = self.train(self.model)
    else:
        self.model.load_weights('D:\\data\\cifar10vgg.h5')

In [ ]:
# 필요한 라이브러리 import 하는 코드 
import numpy as np

from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                                     # 텐서 플로우 명령어 필요할 때 tf 대신에 
                                     # k 를 쓰겠다라는 의미 

from keras.datasets import cifar10

# 데이터 적제 
NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]


#  0 ~ 1사이로 정규화 한다. 
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
# 라벨을 10개의 one encoding 생성한다. 
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)



# 모델 만들기 
input_layer = Input((32,32,3))

x = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# 추가된 convolution 계층
x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# 완전 연결 계층 
x = Flatten()(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

# 추가된 완전 연결 계층
x = Dense(100)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)
x = Dense(NUM_CLASSES)(x)

output_layer = Activation('softmax')(x)

model = Model(input_layer, output_layer)

# convolution 층 4개, 완전 연결계층 2개인 6층 신경망 

model.summary()

# 모델 훈련

model_exists = True

if model_exists:
    
    model.load_weights('D:\\data\\cifar10_v1.h5')
    
    CLASSES = np.array(['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말',
                                '배', '트럭' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test [1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)
    
else:
        
    opt = Adam(lr=0.0005)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(x_train
              , y_train
              , batch_size=100
              , epochs=1
              , shuffle=True
               , validation_split=0.25)
    
    
    model.save_weights('D:\\data\\cifar10_v1.h5')
    
    model.evaluate(x_test, y_test, batch_size=1000)
    
    CLASSES = np.array(['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말',
                                '배', '트럭' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test[1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)

## ■ cifar10 을 vgg16으로 구성된 신경망으로 학습시키기

In [ ]:
# 초기버전 ver1

ROWS = 224
COLS = 224
CHANNELS = 3

def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR) #cv2.IMREAD_GRAYSCALE
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


def prep_data(images): 
    count = len(images)
    data = np.ndarray((count, ROWS, COLS, CHANNELS), dtype=np.uint8) # 4차원 짜리 비어있는 numpy array형을 만듬.
    
    for i, image_file in enumerate(images):
        image = read_image(image_file) # resize된 이미지를 image변수에 저장함
        data[i] = image # data라는 numpy array에 image 변수값을 저장.
        if i%250 == 0: print('Processed {} of {}'.format(i, count)) # 진행경과를 확인하는 코드
        
    return data


x_train = prep_data(train_images)
x_test = prep_data(test_images)

print(x_train[0].shape)

## ■ Kaggle 버젼으로 저장해놓은 cifar10 사진들을 가져옴.

In [ ]:
# 중간버전 ver2
# 케글 버젼으로 저장된 cifar10 사진을 가져옴.

import os, cv2, random
import numpy as np
import pandas as pd
# 필요한 라이브러리 import 하는 코드 
import numpy as np

from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                                     # 텐서 플로우 명령어 필요할 때 tf 대신에 
                                     # k 를 쓰겠다라는 의미 

from keras.datasets import cifar10

TRAIN_DIR = 'D:\\data\\cifar10\\train\\'
TEST_DIR = 'D:\\data\\cifar10\\test\\'

train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] # use this for full dataset
test_images =  [TEST_DIR+i for i in os.listdir(TEST_DIR)]

# slice datasets for memory efficiency on Kaggle Kernels, delete if using full dataset
train_images = test_images[:1000] 
test_images =  test_images[:1000]


ROWS = 224
COLS = 224
CHANNELS = 3

def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR) #cv2.IMREAD_GRAYSCALE
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

def prep_data(images):
    count = len(images) # 전체 사진 갯수 // 현재 : 1000개
    data = np.ndarray((count, ROWS, COLS, CHANNELS), dtype=np.uint8)
    # 4차원 짜리 비어있는 numpy array 생성 

    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image       
        if i%250 == 0: print('Processed {} of {}'.format(i, count))
        
    return data

x_train = prep_data(train_images)
x_test = prep_data(test_images)

print(x_train[0].shape)

## ■ keras 버전 (224,224,3)으로 바꾸어 주는 코드

In [ ]:
# keras ver 224,224,3으로 바꾸는 코드
# keras에 내장되어있는 cifar10 데이터들을 가져옴

import os, cv2, random

# 필요한 라이브러리 import 하는 코드 
import numpy as np

from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                                     # 텐서 플로우 명령어 필요할 때 tf 대신에 
                                     # k 를 쓰겠다라는 의미 

from keras.datasets import cifar10

# 데이터 적제 
NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

train_images= x_train 
test_images = x_test 


ROWS=224
COLS =224
CHANNELS = 3

def read_image(file_path):
    img = file_path
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

def prep_data(images):
    count = len(images)
    data = np.ndarray((count, ROWS, COLS,CHANNELS), dtype=np.uint8)
    # 4차원 짜리 비어있는 numpy array 생성 

    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image       
        if i%250 == 0: print('Processed {} of {}'.format(i, count))
        
    return data

x_train = prep_data(train_images)
x_test = prep_data(test_images)

print(x_train.shape)

## ■ cifar10 을 vgg16으로 구성된 신경망으로 학습시키는 풀코드

In [ ]:
import os, cv2, random

# 필요한 라이브러리 import 하는 코드 
import numpy as np

from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation, MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                                     # 텐서 플로우 명령어 필요할 때 tf 대신에 
                                     # k 를 쓰겠다라는 의미 

from keras.datasets import cifar10

# 데이터 적제 
NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

train_images= x_train 
test_images = x_test 


ROWS= 224
COLS = 224
CHANNELS = 3

def read_image(file_path):
    img = file_path
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

def prep_data(images):
    count = len(images)
    data = np.ndarray((count, ROWS, COLS,CHANNELS), dtype=np.uint8)
    # 4차원 짜리 비어있는 numpy array 생성 

    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image       
        if i%250 == 0: print('Processed {} of {}'.format(i, count))
        
    return data

x_train = prep_data(train_images)
x_test = prep_data(test_images)

x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]


#  0 ~ 1사이로 정규화 한다. 
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
# 라벨을 10개의 one encoding 생성한다. 
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)



# 모델 만들기 
input_layer = Input((224,224,3))

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size=2,strides=2)(x)


x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size=2,strides=2)(x)


x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size=2,strides=2)(x)


x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size=2,strides=2)(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size=2,strides=2)(x)

# 완전 연결 계층 
x = Flatten()(x)

x = Dense(4096)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(4096)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x)

model = Model(input_layer, output_layer)

# convolution 층 4개, 완전 연결계층 2개인 6층 신경망 

model.summary()

# 모델 훈련

model_exists = False


if model_exists:
    
    
    model.load_weights('D:\\data\\cifar10vgg16.h5')
    
    CLASSES = np.array(['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말',
                                '배', '트럭' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test [1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)
    
else:
        
    opt = Adam(lr=0.0005)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(x_train
              , y_train
              , batch_size=32
              , epochs=1
              , shuffle=True
               , validation_data=(x_test,y_test) )
    
    
    model.save_weights('D:\\data\\cifar10vgg16.h5')
    
    model.evaluate(x_test, y_test, batch_size=1000)
    
    CLASSES = np.array(['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말',
                                '배', '트럭' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test[1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)

### ■ 기존의 것은 1000개씩 사진들을 가져오기때문에 10개로 줄여서 돌리는 코드

In [ ]:
# 필요한 라이브러리 import 하는 코드

import numpy as np
import os, cv2, random
from keras.layers import Input, MaxPooling2D, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K
from keras.datasets import cifar10

# 데이터 적재
NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train[:10]
y_train = y_train[:10]
x_test = x_test[:10]
y_test = y_test[:10]

train_images= x_train
test_images = x_test


ROWS=224
COLS =224
CHANNELS = 3

def read_image(file_path):
img = file_path
return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

def prep_data(images):
count = len(images)
data = np.ndarray((count, ROWS, COLS,CHANNELS), dtype=np.uint8)
# 4차원 짜리 비어있는 numpy array 생성

for i, image_file in enumerate(images):
image = read_image(image_file)
data[i] = image
if i%250 == 0: print('Processed {} of {}'.format(i, count))

return data

x_train = prep_data(train_images)
x_test = prep_data(test_images)

print(x_train.shape)

# 0 ~ 1사이로 정규화 한다.
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 라벨을 10개의 one encoding 생성한다.
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

# 모델 만들기
input_layer = Input((224,224,3))

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

# 완전 연결 계층
x = Flatten()(x)

x = Dense(4096)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(4096)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x)

model = Model(input_layer, output_layer)

# convolution 층 5개, 완전 연결계층 3개인 8층 신경망
model.summary()

# 모델 훈련
opt = Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(x_train
, y_train
, batch_size=32
, epochs=10
, shuffle=True
, validation_data = (x_test, y_test))

model.evaluate(x_test, y_test, batch_size=1000)

preds = model.predict(x_test)

CLASSES = np.array(['비행기', '자동차', '새', '고양이', '사슴', \
'개', '개구리', '말', '배', '트럭'])

preds = model.predict(x_test)
preds_value = CLASSES[np.argmax(preds, axis = -1)][2] #axis = -1은 마지막 차원으로 배열을 압축하라는 뜻임
actual_value = CLASSES[np.argmax(y_test, axis = -1)][2] # (10000, 10) → (10000, 1)

print(preds_value)
print(actual_value)

model.save_weights('cifar10_vgg16.h5')